In [35]:
#import relevant libraries
from bs4 import BeautifulSoup #parsing html
import requests #working with urls
import json #working with json data

In [36]:
s = requests.Session()

In [37]:
#get transtats html from url
r = s.get('https://www.transtats.bts.gov/Data_Elements.aspx?Data=2')
#parse html text
soup = BeautifulSoup(r.text, "lxml")

In [38]:
def get_options(soup, tag):
    """
    this function goes through the parsed soup of transtats website
    searches for the specified tag
    returns a list of elements of the tag
    used to obtain list of carriers and airports
    """
    options = []
    get_list = soup.find(id=tag)
    for option in get_list.find_all('option'):
        options.append(option['value'])
    return options

In [39]:
def print_list(label, codes):
    """
    prints label of list and elements of list
    useful for viewing elements of a list
    """
    print("\n%s:" % label)
    for code in codes:
        print(code)

In [40]:
def make_request(viewstate, eventvalidation, carrier, airport):
    """
    request html containing data for specified carrier-airport combination from transtats website
    some elements required for making request are hard-coded, others are given as inputs
    writes the obtained data to a html file
    data obtained from this function will still require further parsing to obtain the required table of information
    """
    url = "https://www.transtats.bts.gov/Data_Elements.aspx?Data=2"
    request_elements = (('__EVENTTARGET', ""), 
                        ('__EVENTARGUMENT', ""), 
                        ('__VIEWSTATE', viewstate), 
                        ('__VIEWSTATEGENERATOR', "8E3A4798"), 
                        ('__EVENTVALIDATION', eventvalidation), 
                        ('CarrierList', carrier), 
                        ('AirportList', airport), 
                        ('Submit', "Submit"))

    r = s.post(url, request_elements)
    filename = carrier+"-"+airport+".html"
    with open(filename, "w") as f:
        f.write(r.text)
    return filename

In [41]:
def extract_data(soup):
    """
    this helper function gets 2 of the required elements for making requests to the transtats website for
    carrier-airport combinations.
    """
    data = {"eventvalidation": "",
            "viewstate": ""}
    vs = soup.find(id="__VIEWSTATE")
    viewstate = vs["value"]
    ev = soup.find(id="__EVENTVALIDATION")
    eventvalidation = ev['value']

    return viewstate, eventvalidation

In [42]:
#get list of carriers and airports using the get_options function
carrier_codes = get_options(soup, 'CarrierList')
airport_codes = get_options(soup, 'AirportList')

In [47]:
#print_list("Carriers", carrier_codes)
#print_list("Airports", airport_codes)

In [48]:
#get request elements required to query the transtats website then query website for the 
#"all carrier"-"all airport" combination, then store resulting data in a html file
viewstate, eventvalidation = extract_data(soup)
all_agg = {}

filename = make_request(viewstate, eventvalidation, carrier_codes[0], airport_codes[0])
print(filename)

All-All.html
